In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,when,sum, explode

In [2]:
spark = SparkSession.builder \
    .appName("Spark XML Example") \
    .config("spark.jars", 'C:\SPARK\spark-xml_2.12-0.16.0.jar') \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:4.4.3") \
    .config("spark.jars.repositories", "https://repo1.maven.org/maven2/") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .master("local[*]") \
    .getOrCreate()

df = spark.read.parquet("parquetFile/final.parquet")

# sorted_df = df.orderBy("word")


In [ ]:
from pyspark.sql.functions import col

# Hàm tìm kiếm dựa trên các DataFrame con theo chữ cái đầu tiên của từ khóa
def search_in_loaded_dfs(query: str):
    # Lấy chữ cái đầu tiên của query (chữ cái đầu sẽ quyết định DataFrame con)
    first_letter = query[0].lower()
    
    # Kiểm tra nếu DataFrame tương ứng với chữ cái đầu tiên có trong dictionary
    if first_letter in dfs_dict:
        # Lấy DataFrame con theo chữ cái đầu tiên
        df = dfs_dict[first_letter]
        
        # Tìm kiếm từ khóa trong cột 'word'
        result_df = df.filter(col("word") == query)
        
        return result_df
    else:
        print(f"Không tìm thấy DataFrame cho chữ cái đầu tiên '{first_letter}'")
        return None

# Giả sử bạn đã có các DataFrame con như a_df, b_df, c_df, ...
# Tạo một dictionary chứa các DataFrame con để tìm kiếm
dfs_dict = {
    "a": a_df,
    "b": b_df,
    "c": c_df,
    # Thêm các DataFrame con tương ứng với các chữ cái khác
}

# Từ khóa cần tìm
query = "computer"

# Tìm kiếm trong DataFrame tương ứng
result_df = search_in_loaded_dfs(query, dfs_dict)

# Hiển thị kết quả tìm kiếm (nếu có)
if result_df is not None:
    result_df.show(truncate=False)


In [4]:
import re

string = "Anarchism-traditional, Anarchist"

# Bước 1: Chuyển tất cả ký tự về chữ thường
string = string.lower()

# Bước 2: Thay thế ký tự đặc biệt bằng khoảng trắng
string = re.sub(r"[^a-zA-Z0-9]", " ", string)

# Bước 3: Tách chuỗi thành danh sách các từ
key_words = string.split()

result_df = df.filter(col("word").isin(key_words))

# Tách cột locations (explode để xử lý từng phần tử)
exploded_df = result_df.withColumn("location_item", explode(col("locations")))

# Trích xuất các trường từ StructType trong cột locations
processed_df = exploded_df.select(
    col("word"),
    col("location_item.doc_id").alias("doc_id"),
    col("location_item.location").alias("position"),
    col("location_item.frequency").alias("count")
)

# Tính điểm dựa trên vị trí
scored_df = processed_df.withColumn(
    "score",
    when(col("position") == "revision_comment", col("count") * 5)
    .when(col("position") == "title", col("count") * 150)
    .otherwise(col("count") * 1)
)

# Tính tổng điểm cho mỗi bài viết (doc_id)
aggregated_df = scored_df.groupBy("doc_id").agg(sum("score").alias("total_score"))

# Sắp xếp theo tổng điểm và lấy top 3 bài viết có điểm cao nhất
top_3_ids = aggregated_df.orderBy(col("total_score").desc()).limit(5)

# Hiển thị kết quả
top_3_ids.show(truncate=False)


+----------------------------+-----------+
|doc_id                      |total_score|
+----------------------------+-----------+
|individualist anarchism     |835        |
|anarchism                   |557        |
|traditional chinese medicine|522        |
|anarchist communism         |353        |
|traditional anarchism       |306        |
+----------------------------+-----------+



In [5]:
spark.stop()